In [1]:
import random

import numpy as np
import polars as pl

random.seed(46)

In [2]:
nn_df = pl.read_csv("/kaggle/input/uspto-boolean-search-optimization/nearest_neighbors.csv")
nn_df.head(2)

publication_number,neighbor_0,neighbor_1,neighbor_2,neighbor_3,neighbor_4,neighbor_5,neighbor_6,neighbor_7,neighbor_8,neighbor_9,neighbor_10,neighbor_11,neighbor_12,neighbor_13,neighbor_14,neighbor_15,neighbor_16,neighbor_17,neighbor_18,neighbor_19,neighbor_20,neighbor_21,neighbor_22,neighbor_23,neighbor_24,neighbor_25,neighbor_26,neighbor_27,neighbor_28,neighbor_29,neighbor_30,neighbor_31,neighbor_32,neighbor_33,neighbor_34,neighbor_35,neighbor_36,neighbor_37,neighbor_38,neighbor_39,neighbor_40,neighbor_41,neighbor_42,neighbor_43,neighbor_44,neighbor_45,neighbor_46,neighbor_47,neighbor_48,neighbor_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-1-A""","""US-189152-A""","""US-1189501-A""","""US-409430-A""","""US-421894-A""","""US-9052-A""","""US-435641-A""","""US-247802-A""","""US-566579-A""","""US-148439-A""","""US-488484-A""","""US-1003894-A""","""US-951842-A""","""US-965846-A""","""US-733217-A""","""US-1681171-A""","""US-339-A""","""US-1206147-A""","""US-1213323-A""","""US-9716-A""","""US-882618-A""","""US-1161812-A""","""US-6321-A""","""US-3267876-A""","""US-1136947-A""","""US-1097810-A""","""US-939423-A""","""US-986261-A""","""US-975048-A""","""US-95968-A""","""US-5532-A""","""US-393896-A""","""US-156138-A""","""US-144271-A""","""US-357649-A""","""US-1182286-A""","""US-1342938-A""","""US-1329037-A""","""US-42039-A""","""US-3237572-A""","""US-462944-A""","""US-1108807-A""","""US-1393935-A""","""US-45450-A""","""US-381847-A""","""US-1354714-A""","""US-661045-A""","""US-737513-A""","""US-447255-A""","""US-1510390-A""","""US-1075814-A"""
"""US-1-P""","""US-93-P""","""US-5247-P""","""US-4994-P""","""US-79-P""","""US-32-P""","""US-1997369-A""","""US-10157-P""","""US-5287-P""","""US-PP13497-P2""","""US-10575-P""","""US-4626-P""","""US-5307-P""","""US-2202966-A""","""US-PP13458-P2""","""US-4462-P""","""US-PP28503-P3""","""US-4274-P""","""US-7619-P""","""US-PP12859-P2""","""US-PP20477-P3""","""US-PP16660-P3""","""US-4953-P""","""US-PP12171-P2""","""US-PP27118-P3""","""US-1975536-A""","""US-PP29050-P2""","""US-4761-P""","""US-PP30184-P2""","""US-4489-P""","""US-2013239269-…","""US-7003-P""","""US-4702-P""","""US-4724-P""","""US-4979-P""","""US-7731-P""","""US-PP28543-P3""","""US-7882-P""","""US-PP21792-P3""","""US-PP31404-P3""","""US-PP28469-P3""","""US-11160-P""","""US-PP27772-P3""","""US-4163-P""","""US-PP13496-P2""","""US-10448-P""","""US-5873-P""","""US-PP23739-P3""","""US-PP21937-P2""","""US-6339-P""","""US-10059-P"""


In [3]:
for i in range(50):
    nn_df = nn_df.rename({f"neighbor_{i}": f"target_{i}"})
print(nn_df.shape)
nn_df.head(2)

(13307647, 51)


publication_number,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,target_10,target_11,target_12,target_13,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23,target_24,target_25,target_26,target_27,target_28,target_29,target_30,target_31,target_32,target_33,target_34,target_35,target_36,target_37,target_38,target_39,target_40,target_41,target_42,target_43,target_44,target_45,target_46,target_47,target_48,target_49
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""US-1-A""","""US-189152-A""","""US-1189501-A""","""US-409430-A""","""US-421894-A""","""US-9052-A""","""US-435641-A""","""US-247802-A""","""US-566579-A""","""US-148439-A""","""US-488484-A""","""US-1003894-A""","""US-951842-A""","""US-965846-A""","""US-733217-A""","""US-1681171-A""","""US-339-A""","""US-1206147-A""","""US-1213323-A""","""US-9716-A""","""US-882618-A""","""US-1161812-A""","""US-6321-A""","""US-3267876-A""","""US-1136947-A""","""US-1097810-A""","""US-939423-A""","""US-986261-A""","""US-975048-A""","""US-95968-A""","""US-5532-A""","""US-393896-A""","""US-156138-A""","""US-144271-A""","""US-357649-A""","""US-1182286-A""","""US-1342938-A""","""US-1329037-A""","""US-42039-A""","""US-3237572-A""","""US-462944-A""","""US-1108807-A""","""US-1393935-A""","""US-45450-A""","""US-381847-A""","""US-1354714-A""","""US-661045-A""","""US-737513-A""","""US-447255-A""","""US-1510390-A""","""US-1075814-A"""
"""US-1-P""","""US-93-P""","""US-5247-P""","""US-4994-P""","""US-79-P""","""US-32-P""","""US-1997369-A""","""US-10157-P""","""US-5287-P""","""US-PP13497-P2""","""US-10575-P""","""US-4626-P""","""US-5307-P""","""US-2202966-A""","""US-PP13458-P2""","""US-4462-P""","""US-PP28503-P3""","""US-4274-P""","""US-7619-P""","""US-PP12859-P2""","""US-PP20477-P3""","""US-PP16660-P3""","""US-4953-P""","""US-PP12171-P2""","""US-PP27118-P3""","""US-1975536-A""","""US-PP29050-P2""","""US-4761-P""","""US-PP30184-P2""","""US-4489-P""","""US-2013239269-…","""US-7003-P""","""US-4702-P""","""US-4724-P""","""US-4979-P""","""US-7731-P""","""US-PP28543-P3""","""US-7882-P""","""US-PP21792-P3""","""US-PP31404-P3""","""US-PP28469-P3""","""US-11160-P""","""US-PP27772-P3""","""US-4163-P""","""US-PP13496-P2""","""US-10448-P""","""US-5873-P""","""US-PP23739-P3""","""US-PP21937-P2""","""US-6339-P""","""US-10059-P"""


In [4]:
metadata = pl.read_parquet(
    "/kaggle/input/uspto-boolean-search-optimization/patent_metadata.parquet"
)
metadata = metadata.with_columns(pl.col("publication_date").dt.year().alias("year"))
val_patents = set(metadata.filter(pl.col("year") >= 1975)["publication_number"].to_list())
print(len(val_patents))

9457935


In [5]:
nn_df = nn_df.filter(pl.col("publication_number").is_in(val_patents))
print(len(nn_df))

9457949


In [6]:
for i in range(10):
    nn_df = nn_df.sample(2500, shuffle=True, seed=i)
    nn_df.write_parquet(f"train2500_seed{i}.parquet")

In [7]:
# nn_df.write_parquet("train_full.parquet")

---

In [8]:
# nn_df = pl.read_csv("/kaggle/input/uspto-boolean-search-optimization/nearest_neighbors.csv")
# for i in range(50):
#     nn_df = nn_df.rename({f"neighbor_{i}": f"target_{i}"})

# metadata = pl.read_parquet(
#     "/kaggle/input/uspto-boolean-search-optimization/patent_metadata.parquet"
# )
# metadata = metadata.with_columns(pl.col("publication_date").dt.year().alias("year"))
# val_patents = set(metadata.filter(pl.col("year") < 1975)["publication_number"].to_list())

# nn_df = nn_df.filter(pl.col("publication_number").is_in(val_patents))
# nn_df.write_parquet("train_old.parquet")